In [6]:
# Imports
import tensorflow.compat.v1 as tf
import tensorflow.compat.v1.keras as K
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()

In [18]:
# 0. Sequential
"""
Write a function that builds a neural network with the Keras library
Use Sequential model since that's in the task name
"""
def build_model_0(nx, layers, activations, lambtha, keep_prob):
    """
    nx: number of input features to the network
    layers: list containing the number of nodes in each layer of the network
    activations: list containing the activation functions for each layer
    lambtha: L2 regularization parameter
    keep_prob: probability that a node will be kept for dropout

    Not allowed to use `Input` class
    Returns the keras model
    """
    model = K.Sequential()
    # Create our input layer
    model.add(K.layers.InputLayer(input_shape=(nx,)))
    # Create all of our hidden layers
    for layer in range(len(layers)):
        model.add(K.layers.Dense(units=layers[layer],
                                 activation=activations[layer],
                                 kernel_regularizer=K.regularizers.l2(lambtha)
                                 ))
        # Handle dropout on hidden layers
        if layer < len(layers) - 1:
            model.add(K.layers.Dropout(1 - keep_prob))
    return model

In [19]:
# 0-main
network = build_model_0(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
network.summary()
print(network.losses)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 256)               200960    
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 256)               65792     
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_18 (Dense)            (None, 10)                2570      
                                                                 
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'dense_16/kernel/Regularizer/mul:0' 

In [20]:
# 0. Sequential | Attempt 2
"""
Write a function that builds a neural network with the Keras library
Use Sequential model since that's in the task name
"""
def build_model_1(nx, layers, activations, lambtha, keep_prob):
    """
    nx: number of input features to the network
    layers: list containing the number of nodes in each layer of the network
    activations: list containing the activation functions for each layer
    lambtha: L2 regularization parameter
    keep_prob: probability that a node will be kept for dropout

    Not allowed to use `Input` class
    Returns the keras model
    """
    model = K.Sequential()
    # Create our input layer
    model.add(K.layers.Dense(units=layers[0],
                             activation = activations[0],
                             kernel_regularizer=K.regularizers.l2(lambtha),
                             input_shape=(nx,)))
    # Create all of our hidden layers
    for layer in range(1, len(layers)):
        if layer <= len(layers) - 1:
            model.add(K.layers.Dropout(1 - keep_prob))
        model.add(K.layers.Dense(units=layers[layer],
                                 activation=activations[layer],
                                 kernel_regularizer=K.regularizers.l2(lambtha)
                                 ))
        # Handle dropout on hidden layers
    return model

In [21]:
# 0-main | Attempt 2
network = build_model_1(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
network.summary()
print(network.losses)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 256)               200960    
                                                                 
 dropout_10 (Dropout)        (None, 256)               0         
                                                                 
 dense_20 (Dense)            (None, 256)               65792     
                                                                 
 dropout_11 (Dropout)        (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                2570      
                                                                 
Total params: 269,322
Trainable params: 269,322
Non-trainable params: 0
_________________________________________________________________
[<tf.Tensor 'dense_19/kernel/Regularizer/mul:0' 

In [4]:
# Task 1. Input
"""
Write a function that builds a neural network with the Keras library
"""
def build_model(nx, layers, activations, lambtha, keep_prob):
    """
    nx: number of input features to the network
    layers: list containing the number of nodes in each layer of the network
    activations: list containing the activation functions for each layer
    lambtha: L2 regularization parameter
    keep_prob: probability that a node will be kept for dropout

    Not allowed to use `Sequential` class
    Returns the keras model
    """
    inputs = K.Input(shape=(nx,))
    x = K.layers.Dense(units=layers[0],
                       activation=activations[0],
                       kernel_regularizer=K.regularizers.l2(lambtha)
                       )(inputs)
    for layer in range(1, len(layers)):
        dropout_layer = K.layers.Dropout(1 - keep_prob)(x)
        x = K.layers.Dense(units=layers[layer],
                           activation=activations[layer],
                           kernel_regularizer=K.regularizers.l2(lambtha)
                           )(dropout_layer)

    model = K.Model(inputs=inputs, outputs=x)
    return model



In [23]:
# 1-main
model = build_model(200, [10], ['tanh'], 0.01, 0.6)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 200)]             0         
                                                                 
 dense_22 (Dense)            (None, 10)                2010      
                                                                 
Total params: 2,010
Trainable params: 2,010
Non-trainable params: 0
_________________________________________________________________


In [11]:
# 2. Optimize
"""
Write a function that sets up Adam optimization for a keras model with
categorical crossentropy loss and accuracy metrics
"""
def optimize_model(network, alpha, beta1, beta2):
    """
    network: the model to optimize
    alpha: learning rate
    beta1: the first Adam optimization parameter
    beta2: second Adam optimization parameter
    Returns none
    """
    Adam_opt = K.optimizers.Adam(lr=alpha,
                                 beta_1=beta1,
                                 beta_2=beta2)
    network.compile(optimizer=Adam_opt,
                    loss="categorical_crossentropy",
                    metrics=['accuracy'])
    return None

In [12]:
# 2-main
model = build_model(784, [256, 256, 10], ['tanh', 'tanh', 'softmax'], 0.001, 0.95)
optimize_model(model, 0.01, 0.99, 0.9)
print(model.loss)
print(model.metrics)
opt = model.optimizer
print(opt.__class__)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run((opt.lr, opt.beta_1, opt.beta_2))) 

categorical_crossentropy
<class 'keras.optimizers.optimizer_v2.adam.Adam'>


/home/bsbanotto/.local/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


FailedPreconditionError: Graph execution error:

Detected at node 'learning_rate_3/Read/ReadVariableOp' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/bsbanotto/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/bsbanotto/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/bsbanotto/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/bsbanotto/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      